In [11]:
import pandas as pd

combined_df = pd.read_csv('../datasets/first_100/accepted_rejected_responses.csv')
combined_df.head()

,Unnamed: 0,Prompt,Accepted_response,Rejected_response
0,0,Corruption involving the contractors is the ch...,Corruption involving the contractors is the ch...,Corruption involving the contractors is the ch...
1,1,5][6][7] The term was coined by the author Maki,5][6][7] The term was coined by the author Mak...,5][6][7] The term was coined by the author Mak...
2,2,"As for the internals, the people with the most...","As for the internals, the people with the most...","As for the internals, the people with the most..."
3,3,"On March 2, 2017, roughly 100 of our","On March 2, 2017, roughly 100 of our members a...","On March 2, 2017, roughly 100 of our members a..."
4,4,"Syrian Democratic Forces’ spokesman, Talal Sil...","Syrian Democratic Forces’ spokesman, Talal Sil...","Syrian Democratic Forces’ spokesman, Talal Sil..."


In [5]:
!pip install trl
!pip install safetensors


  Obtaining dependency information for safetensors from https://files.pythonhosted.org/packages/91/4f/bded5d0435a72f1bd39f5aaa81077d84fa39526a7c055cc8b8aa44fce681/safetensors-0.4.0-cp311-cp311-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.4/425.4 kB 7.3 MB/s eta 0:00:00a 0:00:01


In [6]:
import trl


/Users/saumyabakshi/anaconda3/lib/python3.11/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [12]:
import random
import pandas as pd
from operator import itemgetter
import torch
import warnings
warnings.filterwarnings('ignore')
from datasets import Dataset, load_dataset
from transformers import AutoModelForSequenceClassification,AutoTokenizer,TrainingArguments
from trl import RewardTrainer

In [8]:
#Select a base model whch we need to train for reward modeling.
model_name = "distilroberta-base"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weigh

In [19]:
def formatting_func(examples):
    kwargs = {"padding": "max_length", "truncation": True, "max_length": 512, "return_tensors": "pt"}
    prompt_plus_chosen_response = examples["Accepted_response"]
    prompt_plus_rejected_response = examples["Rejected_response"]
    tokens_chosen = tokenizer.encode_plus(prompt_plus_chosen_response, **kwargs)
    tokens_rejected = tokenizer.encode_plus(prompt_plus_rejected_response, **kwargs)
    return {
        "input_ids_chosen": tokens_chosen["input_ids"][0], "attention_mask_chosen": tokens_chosen["attention_mask"][0],
        "input_ids_rejected": tokens_rejected["input_ids"][0], "attention_mask_rejected": tokens_rejected["attention_mask"][0]
    }

from datasets import Dataset

# Assuming `formatted_dataset` is a DataFrame after applying `formatting_func`
dataset = Dataset.from_pandas(combined_df)

formatted_dataset = dataset.map(formatting_func)

formatted_dataset = formatted_dataset.train_test_split()
# Configuring the training arguments
training_args = TrainingArguments(
    output_dir="./reward_model",
    per_device_train_batch_size=16,
    evaluation_strategy="steps",
    logging_steps=1,
    num_train_epochs = 10,
    report_to=None,
)
# Loading the RewardTrainer from TRL
trainer = RewardTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset["train"],
    eval_dataset=formatted_dataset["test"],
)
trainer.train()

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

  0%|          | 0/50 [00:00<?, ?it/s]

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Could not estimate the number of tokens of the input, floating-point operations will not be computed


{'loss': 0.6869, 'learning_rate': 4.9e-05, 'epoch': 0.2}


  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.7732922434806824, 'eval_accuracy': 0.72, 'eval_runtime': 12.5325, 'eval_samples_per_second': 1.995, 'eval_steps_per_second': 0.319, 'epoch': 9.8}
{'loss': 0.0022, 'learning_rate': 0.0, 'epoch': 10.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.7734549641609192, 'eval_accuracy': 0.72, 'eval_runtime': 12.546, 'eval_samples_per_second': 1.993, 'eval_steps_per_second': 0.319, 'epoch': 10.0}
{'train_runtime': 3316.1998, 'train_samples_per_second': 0.226, 'train_steps_per_second': 0.015, 'train_loss': 0.2663334091147408, 'epoch': 10.0}


TrainOutput(global_step=50, training_loss=0.2663334091147408, metrics={'train_runtime': 3316.1998, 'train_samples_per_second': 0.226, 'train_steps_per_second': 0.015, 'train_loss': 0.2663334091147408, 'epoch': 10.0})

In [ ]:
# TODO add testing code here


In [ ]:
# TODO metrics to see efiicency of model 